In [15]:
import pandas as pd
import matplotlib as plt

In [16]:
df = pd.read_csv('starts.csv')
volunteers = pd.read_csv('volunteers.csv')
df['event_date'] = pd.to_datetime(df['event_date'])
volunteers['event_date'] = pd.to_datetime(volunteers['event_date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178857 entries, 0 to 178856
Data columns (total 23 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   record_id                  178857 non-null  int64         
 1   athlete_id                 178857 non-null  int64         
 2   event_name                 178857 non-null  object        
 3   event_date                 178857 non-null  datetime64[ns]
 4   city                       178857 non-null  object        
 5   region                     178857 non-null  object        
 6   finish_time                178857 non-null  int64         
 7   athlete_gender             178857 non-null  object        
 8   athlete_age                178857 non-null  int64         
 9   is_newbie                  178857 non-null  int64         
 10  temp                       178857 non-null  float64       
 11  feels_like                 178857 non-null  float64 

In [17]:
volunteers = volunteers[volunteers.athlete_age < 100]
df = df[df.athlete_age < 80]
df = df[df.finish_time < 5000]

In [18]:
cnt_ludei = df.groupby(['event_name', 'event_date', 'region']).agg(
    cnt_runners = ('athlete_id', 'count')
)

In [19]:
df.head()

,record_id,athlete_id,event_name,event_date,city,region,finish_time,athlete_gender,athlete_age,is_newbie,...,humidity,dew_point,clouds,wind_speed,wind_deg,weather.main,weather.description,rain.1h,snow.1h,track_description_surface
0,6490,790055465,Нижний пруд,2023-03-25,Калининград,Калининградская область,1260,мужской,43,1,...,90,6.36,25,3.67,223,Rain,light rain,0.10,NaN,"асфальт, бетон"
1,7459,790055465,Нижний пруд,2023-05-06,Калининград,Калининградская область,1098,мужской,43,1,...,54,-1.63,95,4.18,83,Clouds,overcast clouds,NaN,NaN,"асфальт, бетон"
2,10674,790055465,Нижний пруд,2023-09-09,Калининград,Калининградская область,1138,мужской,44,1,...,85,15.06,100,0.80,155,Clouds,overcast clouds,NaN,NaN,"асфальт, бетон"
3,13267,790055465,Нижний пруд,2023-12-09,Калининград,Калининградская область,1420,мужской,44,1,...,95,-5.17,100,3.47,140,Clouds,overcast clouds,NaN,NaN,"асфальт, бетон"
4,9004,790055474,Битца,2023-07-08,Москва,Москва,2288,женский,35,0,...,93,17.59,97,3.50,351,Rain,light rain,0.41,NaN,грунт


In [20]:
df['is_male'] = df['athlete_gender'].apply(lambda x: 1 if x == 'мужской' else 0)
df = df[df['athlete_age'] > 14]

volunteer_counts = volunteers.groupby(['event_date','event_name', 'volunteer_role_name']).size().unstack(fill_value=0)  
df = df.merge(volunteer_counts, on=['event_date','event_name'], how='left')  

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167646 entries, 0 to 167645
Data columns (total 51 columns):
 #   Column                             Non-Null Count   Dtype         
---  ------                             --------------   -----         
 0   record_id                          167646 non-null  int64         
 1   athlete_id                         167646 non-null  int64         
 2   event_name                         167646 non-null  object        
 3   event_date                         167646 non-null  datetime64[ns]
 4   city                               167646 non-null  object        
 5   region                             167646 non-null  object        
 6   finish_time                        167646 non-null  int64         
 7   athlete_gender                     167646 non-null  object        
 8   athlete_age                        167646 non-null  int64         
 9   is_newbie                          167646 non-null  int64         
 10  temp                

In [22]:

# # Создаём новую колонку lines, заполняем её нулями
# df['lines'] = 0

# # Группируем по record_id
# for record_id, group in df.groupby('record_id'):
#     # Сортируем внутри группы по finish_time
#     group = group.sort_values('finish_time')
    
#     # Считаем разницу между соседними записями
#     group['time_diff'] = group['finish_time'].diff()
    
#     # Подсчитываем количество людей с разницей ≤ 15 секунд
#     count = (group['time_diff'] <= 15).sum()
    
#     # Если таких людей больше, чем "Сканирование штрих-кодов", обновляем lines
#     if count > group['Сканирование штрих-кодов'].iloc[0]:  # Берём значение из первой строки группы
#         df.loc[df['record_id'] == record_id, 'lines'] = 1

# # Удаляем временную колонку time_diff (если не нужна)
# df = df.drop(columns=['time_diff'], errors='ignore')

# # Вывод результата
# print(df.head(1))


In [23]:
# df['lines'].value_counts()

In [25]:
# import pandas as pd

# # Пример данных
# data = {
#     'record_id': [1, 1, 1, 2, 2, 2],
#     'finish_time': [10, 20, 35, 50, 60, 70],  # Время в секундах
#     'Сканирование штрих-кодов': [2, 2, 2, 1, 1, 1],
# }

# df = pd.DataFrame(data)

# Функция для вычисления значения 'lines' для каждой группы record_id
def calculate_lines(group):
    # Сортируем внутри группы по finish_time
    group = group.sort_values('finish_time')
    
    # Считаем разницу между соседними значениями finish_time
    group['time_diff'] = group['finish_time'].diff()
    
    # Подсчитываем количество пар с разницей <= 15 секунд
    count_pairs = (group['time_diff'] <= 15).sum()
    
    # Определяем, если count_pairs > "Сканирование штрих-кодов" (берём значение из первой строки группы)
    lines_value = 1 if count_pairs > group['Сканирование штрих-кодов'].iloc[0] else 0
    
    # Удаляем временную колонку time_diff перед возвратом
    group = group.drop(columns=['time_diff'])
    
    # Добавляем колонку lines со значением для всей группы
    group['lines'] = lines_value
    
    return group

# Применяем функцию к каждой группе record_id
df = df.groupby('record_id').apply(calculate_lines).reset_index(drop=True)

# Вывод результата
print(df['lines'].value_counts())


/var/folders/nv/y1xwhy_x1gx4gn0wks7mdjqc0000gn/T/ipykernel_56510/1059586235.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('record_id').apply(calculate_lines).reset_index(drop=True)


lines
1    163313
0      4333
Name: count, dtype: int64
